# Key Index

## Мониторинг ключевых индикаторов финансового и фондового рынков

Есть вопросы? Дмитрий Ходыкин: dmitriy_hodikin (skype)

## Пространство имен

In [1]:
# Импорты

# Базы данных
import psycopg2 as psql

## Дата-время
from datetime import datetime
from datetime import timedelta

## Информация о рынках
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data

## Регистрационные данные
from auth import auth

## Таблицы и вычисления
import pandas as pd
import numpy as np

## Загрузка данных за период

Отслеживаемые индикаторы:
- `S&P 500` — фондовый индекс, в корзину которого включено 505 избранных торгуемых на фондовых биржах США публичных компаний, имеющих наибольшую капитализацию. Список принадлежит компании Standard & Poor’s и ею же составляется.

- `Индекс ММВБ` В Индекс Московской биржи входят 42 компании, акции которых торгуются на российской площадке. Индекс Мосбиржи на 16,23% состоит из бумаг «Лукойла», на 14,03% из Сбербанка, на 12,2% из «Газпрома» и так далее. Точная информация о весе каждой из компаний регулярно публикуется на сайте moex.com.

- `NASDAQ` (сокр. от англ. National Association of Securities Dealers Automated Quotation, читается как «Насдак»  Служба автоматизированных котировок Национальной ассоциации дилеров по ценным бумагам) — американская биржа, специализирующаяся на акциях высокотехнологичных компаний.

- `Russell 2000` - это индекс, измеряющий динамику около 2000 компаний малой капитализации. В индекс входят компании финансового сектора, сферы здравоохранения, потребительских товаров, технологические компании. Russell 2000 является хорошим показателем ситуации на американском рынке, т.к. состоит из средних и маленьких компаний США, которые более волатильны и имеют больший потенциал роста, чем крупные по капитализации компании.

- `Казначейские облигации США` на 30/10/5 лет, а также на 13 недель. Превышение доходности краткосрочных облигаций над долгосрочными (10-30 лет) говорит об инверсии доходности и надвигающейся рецессии, т.к. текущие риски превышают будущие риски инвесторов. Все 8-мь рецессий в США совпадали с проявлением эффекта инверсии доходности.

- `Стоимость товаров` таких как: нефть, природный газ, золото.

In [2]:
# Тикеры ключевых индикаторов

tickers = [
    'RUB=X',     # Курс доллара к рублю
    'EURRUB=X',  # Курс евро к рублю
    'EURUSD=X',  # Курс евро к доллару
    'CNY=X',     # Курс доллара к юаню
    
    '^GSPC',     # Индекс S&P 500
    'IMOEX.ME',  # Индекс ММВБ
    '^IXIC',     # Индекс NASDAQ
    '^RUT',      # Индекс Russell 2000
    
    '^TYX',      # 30-ти летние казначейсие облигации
    '^TNX',      # 10-ти летние казначейсие облигации
    '^FVX',      # 5-ти летние казначейсие облигации
    '^IRX',      # 13-ти недельные казначейские облигации
    
    'CL=F',      # Сырая нефть (NY Товарная биржа)
    'NG=F',      # Природный газ (NY Товарная биржа)
    'GC=F',      # Золото (COMEX Delayed Price)
    'SI=F'       # Серебро
]

In [3]:
def create_dataset(tickers_list, interval,
                   start_date, end_date):
    """Возвращает датасет по списку тикеров из
    [ticker_list] с интервалом значений временного ряда:
    [1d - день], [1wk - неделя], [1mo - месяц]
    между [start] и [end] date"""
    
    data = pd.DataFrame()
    i = 0
    
    while i < len(tickers_list):
        try:
            r = get_data(
                tickers_list[i],
                start_date = start_date,
                end_date = end_date,
                index_as_date = False,
                interval = interval
            )
            
            data = data.append(r)
            i = i + 1
    
        except BaseException:
            i = i + 1  # Переход к следующему тикеру
            pass

    return data

In [4]:
def get_max_dt():
    """Возвращает максимальную дату, 
    содержащуюся в базе данных"""
    
    # Авторизация в базе данных
    conn = psql.connect(
        dbname=auth.psql_db,
        user=auth.psql_user,
        password=auth.psql_passwd,
        host='localhost'
    )

    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT MAX (dt)
        FROM exrates;
        """
    )
    # Выполнение запроса
    conn.commit()
    # Вывод данных запроса
    max_dt = cursor.fetchall()
    max_dt, = max_dt[0]

    cursor.close()
    conn.close()
    
    return max_dt

In [5]:
# Определение отчетного периода

# Период в базе данных
max_dt = get_max_dt()
start_date = max_dt + timedelta(days=1)
start_date = start_date.strftime("%Y-%m-%d")
# Текущий период
now = datetime.now()
end_date = now.strftime("%Y-%m-%d")

print('Начальная дата:', start_date,
     'Конечная дата:', end_date)

Начальная дата: 2020-12-29 Конечная дата: 2020-12-29


In [6]:
# Сформируем датасет по тикерам

data = create_dataset(
    tickers,
    '1d',
    start_date,
    end_date
)

%time

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.2 µs


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 0
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      7 non-null      datetime64[ns]
 1   open      7 non-null      float64       
 2   high      7 non-null      float64       
 3   low       7 non-null      float64       
 4   close     7 non-null      float64       
 5   adjclose  7 non-null      float64       
 6   volume    7 non-null      int64         
 7   ticker    7 non-null      object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 504.0+ bytes


In [8]:
# Удалим пустые строки

data = data[data['adjclose'] > 0]
data.head()

,date,open,high,low,close,adjclose,volume,ticker
0,2020-12-29,74.127800,74.351097,73.339996,73.559998,73.559998,0,RUB=X
0,2020-12-29,90.507301,90.817001,89.829201,90.164299,90.164299,0,EURRUB=X
0,2020-12-29,1.221598,1.226843,1.221300,1.226091,1.226091,0,EURUSD=X
0,2020-12-29,6.537000,6.537000,6.525100,6.529000,6.529000,0,CNY=X
0,2020-12-28,3723.030029,3740.510010,3723.030029,3735.360107,3735.360107,1651103623,^GSPC


In [9]:
# Работа с БД

def insert_into_db(data):
    """Вставляет переданные в функцию данные в БД"""

    # Авторизация в базе данных
    conn = psql.connect(
        dbname=auth.psql_db,
        user=auth.psql_user,
        password=auth.psql_passwd,
        host='localhost'
    )

    cursor = conn.cursor()
    print('Создано подключение к БД:', auth.psql_db)
    print('Выполнение запроса...')    

    for index, row in data.iterrows():
        values = (
            f"{row['date']}", 
            row['open'], 
            row['high'],
            row['low'], 
            row['close'], 
            row['adjclose'], 
            row['volume'], 
            row['ticker']
        )

        cursor.execute(
            """
            INSERT INTO 
            exrates (dt, opn, hi, low, cls, adj, vol, ticker) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, values
        )

    conn.commit()

    cursor.close()
    print('Подключение к БД завершено')

    conn.close()
    print('Соединение с БД закрыто')

In [10]:
# Сохранение полученных данных в БД

insert_into_db(data)
%time

Создано подключение к БД: etl
Выполнение запроса...
Подключение к БД завершено
Соединение с БД закрыто
CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 14.5 µs
